In [89]:
import pandas as pd
import numpy as np

In [42]:
filename = '../Downloads/CaseyJones2.mid.txt'

lines = []
with open(filename, 'r') as file:
    lines = file.readlines()
    
lines = pd.DataFrame([list(filter(None, x.strip().split(' '))) for x in lines])
lines = lines[lines[1].str.contains('On').fillna(False)]
lines[0] = lines[0].astype(int).apply(lambda i: round(float(i) / 30))
lines[2] = lines[2].str.replace('ch=', '').astype(int)
lines[3] = lines[3].str.replace('note=', '')
lines[4] = lines[4].str.replace('vol=', '').astype(int)
lines = lines.sort_values(by=[0]).drop([1, 5, 6, 7, 8, 9], axis=1).rename(columns={0: 'eighths', 2: 'channel', 3: 'note', 4: 'vol'}).drop_duplicates()

In [276]:
lines_sorted = lines.sort_values(by=['channel', 'note', 'eighths']).reset_index(drop=True)
end_lines_sorted = lines_sorted[['eighths', 'vol']].iloc[1:].reset_index(drop=True)
end_lines_sorted.columns = ['end_' + c for c in end_lines_sorted]

joined = pd.concat([lines_sorted, end_lines_sorted], axis=1)
joined = joined[(joined['vol'] > 0) & (joined['end_vol'] == 0)]
joined['duration'] = (joined['end_eighths'] - joined['eighths']).astype(int)
joined = joined.sort_values(['channel', 'eighths']).reset_index(drop=True)

overlaps = pd.concat([joined['end_eighths'], joined['eighths'].iloc[1:].reset_index(drop=True)], axis=1)
overlaps['overlap'] = (overlaps['end_eighths'] > overlaps['eighths'])#.rolling(2).apply(lambda o: o.iloc[0] if not o.iloc[1] else o.iloc[1], raw=False).fillna(0).astype(bool)

joined = pd.concat([joined, overlaps['overlap']], axis=1)

In [286]:
a = []
for index, row in joined.iterrows():
    for i in range(row['duration']):
        r = row.copy()
        r.at['eighths'] = r['eighths'] + i
        a.append(r)
        
joined_all = pd.DataFrame(a).reset_index().rename(columns={'index': 'note_id'})

In [287]:
joined_all

,note_id,eighths,channel,note,vol,end_eighths,end_vol,duration,overlap
0,0,64,1,c6,85,68.0,0.0,4,False
1,0,65,1,c6,85,68.0,0.0,4,False
2,0,66,1,c6,85,68.0,0.0,4,False
3,0,67,1,c6,85,68.0,0.0,4,False
4,1,68,1,d6,85,72.0,0.0,4,False
5,1,69,1,d6,85,72.0,0.0,4,False
6,1,70,1,d6,85,72.0,0.0,4,False
7,1,71,1,d6,85,72.0,0.0,4,False
8,2,72,1,d#6,85,76.0,0.0,4,False
9,2,73,1,d#6,85,76.0,0.0,4,False


In [302]:
def channel(x):
    return ','.join(x)

joined_all.pivot_table(index=['eighths', 'end_eighths'], columns='channel', values='note', aggfunc=[channel])

channel               
channel                   1         2    3
eighths end_eighths                       
0       2.0             NaN     e5,g5  NaN
        4.0             NaN       NaN   g3
1       2.0             NaN     e5,g5  NaN
        4.0             NaN       NaN   g3
2       4.0             NaN        d5   g3
3       4.0             NaN        d5   g3
4       8.0             NaN        c5   a3
5       8.0             NaN        c5   a3
6       8.0             NaN        c5   a3
7       8.0             NaN        c5   a3
8       12.0            NaN     d5,g5   c4
9       12.0            NaN     d5,g5   c4
10      12.0            NaN     d5,g5   c4
11      12.0            NaN     d5,g5   c4
12      16.0            NaN       NaN   g4
13      16.0            NaN       NaN   g4
14      16.0            NaN       NaN   g4
15      16.0            NaN       NaN   g4
16      18.0            NaN    d#5,g5  NaN
17      18.0            NaN    d#5,g5  NaN
18      20.0            NaN        d5  NaN
19      20.0            NaN        d5  NaN
20      24.0            NaN        c5   e4
21      24.0            NaN        c5   e4
22      24.0            NaN        c5   e4
23      24.0            NaN        c5   e4
24      28.0            NaN        a4   d4
25      28.0            NaN        a4   d4
26      28.0            NaN        a4   d4
27      28.0            NaN        a4   d4
...                     ...       ...  ...
1800    1808.0          NaN  b4,d5,f5   g3
1801    1803.0           b5       NaN  NaN
        1808.0          NaN  b4,d5,f5   g3
1802    1803.0           b5       NaN  NaN
        1808.0          NaN  b4,d5,f5   g3
1803    1808.0           c6  b4,d5,f5   g3
1804    1808.0           c6  b4,d5,f5   g3
1805    1808.0           c6  b4,d5,f5   g3
1806    1808.0           c6  b4,d5,f5   g3
1807    1808.0           c6  b4,d5,f5   g3
1808    1810.0           e6       NaN  NaN
        1824.0          NaN     c5,e5   c4
1809    1810.0           e6       NaN  NaN
        1824.0          NaN     c5,e5   c4
1810    1812.0           d6       NaN  NaN
        1824.0          NaN     c5,e5   c4
1811    1812.0           d6       NaN  NaN
        1824.0          NaN     c5,e5   c4
1812    1824.0           c6     c5,e5   c4
1813    1824.0           c6     c5,e5   c4
1814    1824.0           c6     c5,e5   c4
1815    1824.0           c6     c5,e5   c4
1816    1824.0           c6     c5,e5   c4
1817    1824.0           c6     c5,e5   c4
1818    1824.0           c6     c5,e5   c4
1819    1824.0           c6     c5,e5   c4
1820    1824.0           c6     c5,e5   c4
1821    1824.0           c6     c5,e5   c4
1822    1824.0           c6     c5,e5   c4
1823    1824.0           c6     c5,e5   c4

[2894 rows x 3 columns]

In [169]:
joined.loc[overlaps.index]

,eighths,channel,note,vol,end_eighths,end_vol,duration
58,484,1,a5,85,492.0,0.0,8.0
254,1812,1,c6,85,1824.0,0.0,12.0
255,0,2,e5,85,2.0,0.0,2.0
259,8,2,d5,85,12.0,0.0,4.0
261,16,2,d#5,85,18.0,0.0,2.0
269,44,2,c5,85,64.0,0.0,20.0
271,64,2,c5,85,80.0,0.0,16.0
273,84,2,c5,85,88.0,0.0,4.0
275,96,2,a4,85,112.0,0.0,16.0
277,116,2,a#4,85,120.0,0.0,4.0
